# NBA Project EDA
Notebook 1

## Purpose

The goal for this notebook is to conduct an exploratory data analysis (EDA) on datasets related to NBA games and players.<br>
The primary objectives include:

1. Setting Up the Environment: 
- Initializing the necessary libraries and configuring the workspace for data exploration.
2.	Exploring the Database: 
- Diving into multiple tables within an NBA-related SQLite database, such as Game Table, Game Summary Table, Player Table, and others. 
- The aim is to understand the structure and contents of these tables, uncover patterns, and identify key statistics that can provide insights into NBA games and player performance.
3.	Loading and Transforming Data: 
- Importing specific datasets from the SQLite database and performing various transformations such as type conversions, data imputation, and cleaning. 
- Feature engineering steps are also applied to enhance the dataset by creating new features like game duration, location info, and attendance metrics.
4.	Merging Data: 
- Combining various datasets to create a comprehensive dataset that can be used for further analysis or predictive modeling. 
- Additional feature engineering is conducted during this phase to incorporate features like the distance between teams, previous season stats, and more.
5.	Exporting Data: 
- After all transformations and merging processes are completed, the resulting datasets are exported for potential use in further analysis or modeling.

## Table of Contents
  - [Setup](#setup)
  - [Exploring Database](#exploring-database)
    - [Game Table](#game-table)
    - [Game Summary Table](#game-summary-table)
    - [Other Stats Table](#other-stats-table)
    - [Officials Table](#officials-table)
    - [Inactive Players Table](#inactive-players-table)
    - [Game Info Table](#game-info-table)
    - [Line Score Table](#line-score-table)
    - [Play by Play Table](#play-by-play-table)
    - [Player Table](#player-table)
    - [Team Table](#team-table)
    - [Common Player Info Table](#common-player-info-table)
    - [Team Details Table](#team-details-table)
    - [Team History Table](#team-history-table)
    - [Draft Combine Stats Table](#draft-combine-stats-table)
    - [Draft History Table](#draft-history-table)
    - [Team Info Common Table](#team-info-common-table)
  - [Loading Select SQLite Data](#loading-select-sqlite-data)
  - [Transforming Data](#transforming-data)
    - [Type Conversions](#type-conversions)
    - [Imputing Data](#imputing-data)
    - [Cleaning Data](#cleaning-data)
    - [Feature Engineering](#feature-engineering)
    - [Exporting Transformed Data (Optional)](#exporting-transformed-data-optional)
  - [Merging Data](#merging-data)
    - [Additional Feature Engineering](#additional-feature-engineering)
  - [Exporting All Data](#exporting-all-data)
  - [Close Database Connection](#close-database-connection)

## Setup

In [5]:
# Package Imports
import sqlite3
import pandas as pd

import time
from geopy.geocoders import Nominatim
from geopy.distance import distance

In [3]:
# Pandas Options
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 1000)

SQLite dataset can be downloaded from Kaggle [here](https://www.kaggle.com/datasets/wyattowalsh/basketball) and save it to "data" folder in present directory

In [3]:
# Defining database location 
db_path = "data/nba.sqlite" 

In [1]:
# Global connection object to connect to SQLite data bases
conn = None

def init_connection(db_path):
    global conn
    if conn is None:
        conn = sqlite3.connect(db_path)
        print("Database connection opened.")
    else:
        print("Database connection is already open.")

def close_connection():
    global conn
    if conn:
        conn.close()
        conn = None
        print("Database connection closed.")

In [6]:
# Initialize the connection with SQLite Database
init_connection(db_path)

Database connection opened.


Convenience Functions

In [113]:
def get_table_shape(table_name):
    """
    Returns the shape (number of rows and columns) of a specified table in the SQLite database.

    Args:
    table_name (str): Name of the table to get the shape of.

    Returns:
    tuple: A tuple containing the number of rows and columns in the table.
    """
    cursor = conn.cursor()
    
    # SQL query to count the number of rows in the table
    row_count_query = f'SELECT COUNT(*) FROM {table_name}'
    cursor.execute(row_count_query)
    row_count = cursor.fetchone()[0]
    
    # Get the column names for the table to determine the number of columns
    columns = get_column_names(cursor, table_name)
    col_count = len(columns)
    
    return (row_count, col_count)  # Return the number of rows and columns as a tuple

def list_tables():
    """
    Lists all tables in the SQLite database.

    Returns:
    list: List of table names.
    """
    cursor = conn.cursor()  # Create a cursor object
    tables = get_table_names(cursor)  # Get all table names
    return tables  # Return the list of table names

def list_columns(table_name):
    """
    Lists all columns in the SQLite database.

    Args:
    table_name (str): Name of the table.

    Returns:
    list: List of column names.
    """
    cursor = conn.cursor()  # Create a cursor object
    columns = get_column_names(cursor, table_name)  # Get all column names
    return columns  # Return the list of column names

def get_table_names(cursor):
    """
    Retrieves all table names in the SQLite database.

    Args:
    cursor (sqlite3.Cursor): SQLite cursor object.

    Returns:
    list: List of table names.
    """
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")  # Execute SQL query to get all table names
    return [row[0] for row in cursor.fetchall()]  # Fetch and return the table names as a list

def get_column_names(cursor, table_name):
    """
    Retrieves all column names for a given table in the SQLite database.

    Args:
    cursor (sqlite3.Cursor): SQLite cursor object.
    table_name (str): Name of the table.

    Returns:
    list: List of column names.
    """
    cursor.execute(f"PRAGMA table_info({table_name});")  # Execute SQL query to get column info for the table
    return [row[1] for row in cursor.fetchall()]  # Fetch and return the column names as a list

#------------------Functions to check for Nulls and Duplicates------------------------#
def check_null_values(table_name):
    """
    Checks for null values in a specified table and returns the result as a pandas DataFrame.

    Args:
    table_name (str): Name of the table to check for null values.

    Returns:
    pd.DataFrame: DataFrame containing the count of null values for each column with non-zero null counts.
    """
    cursor = conn.cursor()
    
    # Get column names for the table
    columns = get_column_names(cursor, table_name)
    
    # SQL query to count null values for each column
    columns_null_count = ', '.join([f'SUM(CASE WHEN {col} IS NULL THEN 1 ELSE 0 END) AS null_{col}' for col in columns])
    null_check_query = f'''
    SELECT 
        {columns_null_count}
    FROM {table_name}
    '''
    
    # Execute the query
    cursor.execute(null_check_query)
    result = cursor.fetchall() # Fetch the result
    
    # Convert the result to a pandas DataFrame
    df = pd.DataFrame(result, columns=[f'{col}' for col in columns])
    
    # Filter out columns with zero null values
    df = df.loc[:, (df != 0).any(axis=0)]
    
    return df  # Return the DataFrame with only non-zero null counts

def count_duplicate_entries(table_name):
    """
    Returns the count of duplicate entries in a specified table.

    Args:
    table_name (str): Name of the table to check for duplicate entries.

    Returns:
    int: Count of duplicate entries in the table.
    """
    cursor = conn.cursor()
    
    # Get column names for the table
    columns = get_column_names(cursor, table_name)
    
    # SQL query to count duplicate entries
    columns_str = ', '.join(columns)
    count_duplicates_query = f'''
    SELECT COUNT(*)
    FROM (
        SELECT {columns_str}
        FROM {table_name}
        GROUP BY {columns_str}
        HAVING COUNT(*) > 1
    ) AS duplicates
    '''
    
    # Execute the query and fetch the count of duplicate entries
    cursor.execute(count_duplicates_query)
    result = cursor.fetchone()
    
    return result[0]  # Return the count of duplicate entries

def get_duplicate_rows(table_name):
    """
    Identifies duplicate rows in a specified table and returns them as a pandas DataFrame.

    Args:
    table_name (str): Name of the table to check for duplicate rows.

    Returns:
    pd.DataFrame: DataFrame containing the duplicate rows.
    """
    cursor = conn.cursor()
    
    # Get column names for the table
    columns = get_column_names(cursor, table_name)
    
    # SQL query to identify duplicate rows
    columns_str = ', '.join(columns)
    duplicate_rows_query = f'''
    SELECT {columns_str}
    FROM {table_name}
    GROUP BY {columns_str}
    HAVING COUNT(*) > 1
    '''
    
    # Execute the query and fetch the duplicate rows
    cursor.execute(duplicate_rows_query)
    result = cursor.fetchall()
    
    # Convert the result to a pandas DataFrame
    df = pd.DataFrame(result, columns=columns)
    
    return df  # Return the DataFrame containing duplicate rows

#------------------- Data Dates -------------------------#
def get_date_range(table_name: str, column_name: str):
    """
    Retrieves the date range (minimum and maximum date) from a specified date column in a given table.

    Parameters:
    table_name (str): The name of the table from which to retrieve the date range.
    column_name (str): The name of the column containing date values.

    Returns:
    tuple: A tuple containing two date objects:
        - The first element is the earliest date (minimum value) in the specified column.
        - The second element is the latest date (maximum value) in the specified column.

    Example:
    --------
    >>> date_range = get_date_range('sqlite_table', 'date_column')
    >>> print(date_range)
    (datetime.datetime(2020, 1, 1, 0, 0), datetime.datetime(2024, 8, 25, 0, 0))

    Notes:
    ------
    - The function assumes that the date values in the specified column are stored as text in the format '%Y-%m-%d %H:%M:%S'.
    """
    # Create a cursor object
    cursor = conn.cursor()
    
    # Create the SQL query to find the min and max dates
    query = f"""
    SELECT MIN({column_name}) as min_date, MAX({column_name}) as max_date
    FROM {table_name}
    """
    
    # Execute the query
    cursor.execute(query)
    
    # Fetch the result
    result = cursor.fetchone()
    
    # Convert the min and max dates to datetime objects
    min_date = result[0]
    max_date = result[1]
    
    # Close the cursor
    cursor.close()
    
    return (min_date, max_date)

Data Transformation Functions

In [9]:
# Function to get latitude and longitude with a pause
def get_coordinates(city):
    try:
        location = geolocator.geocode(city)
        time.sleep(2)  # Pause for 2 second between requests
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        try: # Try a second time if first try fails to connect
            time.sleep(10)  # Pause for 10 second between requests
            location = geolocator.geocode(city)
            return location.latitude, location.longitude
        except Exception as e:
            return None, None
    
# Function to calculate distance between two coordinates
def calculate_distance(row):
    home_coords = (row['latitude_home_team'], row['longitude_home_team'])
    away_coords = (row['latitude_away_team'], row['longitude_away_team'])
    return distance(home_coords, away_coords).miles

# Function to categorize game duration
def game_duration(min):
    if min == 0:
        return 'Regulation'
    elif min < 240:
        return "Ended Early"
    elif min == 240:
        return "Regulation"
    elif min == 265:
        return "Overtime"
    elif min > 265:
        return "Extra Overtime"
    else:
        return "Unknown"

## Exploring Database 

In [10]:
# Verify Database Connection 
sql_query = '''
SELECT * 
FROM common_player_info 
LIMIT 10;
'''
# Read sqlite query results into a pandas DataFrame
df = pd.read_sql_query(sql_query, conn)
# Verify that result of SQL query is stored in the dataframe
df.head()

,person_id,first_name,last_name,display_first_last,display_last_comma_first,display_fi_last,player_slug,birthdate,school,country,last_affiliation,height,weight,season_exp,jersey,position,rosterstatus,games_played_current_season_flag,team_id,team_name,team_abbreviation,team_code,team_city,playercode,from_year,to_year,dleague_flag,nba_flag,games_played_flag,draft_year,draft_round,draft_number,greatest_75_flag
0,76001,Alaa,Abdelnaby,Alaa Abdelnaby,"Abdelnaby, Alaa",A. Abdelnaby,alaa-abdelnaby,1968-06-24 00:00:00,Duke,USA,Duke/USA,6-10,240,5.0,30,Forward,Inactive,N,1610612757,Trail Blazers,POR,blazers,Portland,HISTADD_alaa_abdelnaby,1990.0,1994.0,N,Y,Y,1990,1,25,N
1,76002,Zaid,Abdul-Aziz,Zaid Abdul-Aziz,"Abdul-Aziz, Zaid",Z. Abdul-Aziz,zaid-abdul-aziz,1946-04-07 00:00:00,Iowa State,USA,Iowa State/USA,6-9,235,10.0,54,Center,Inactive,N,1610612745,Rockets,HOU,rockets,Houston,HISTADD_zaid_abdul-aziz,1968.0,1977.0,N,Y,Y,1968,1,5,N
2,76003,Kareem,Abdul-Jabbar,Kareem Abdul-Jabbar,"Abdul-Jabbar, Kareem",K. Abdul-Jabbar,kareem-abdul-jabbar,1947-04-16 00:00:00,UCLA,USA,UCLA/USA,7-2,225,20.0,33,Center,Inactive,N,1610612747,Lakers,LAL,lakers,Los Angeles,HISTADD_kareem_abdul-jabbar,1969.0,1988.0,N,Y,Y,1969,1,1,Y
3,949,Shareef,Abdur-Rahim,Shareef Abdur-Rahim,"Abdur-Rahim, Shareef",S. Abdur-Rahim,shareef-abdur-rahim,1976-12-11 00:00:00,California,USA,California/USA,6-9,245,13.0,3,Forward,Inactive,N,1610612763,Grizzlies,VAN,grizzlies,Vancouver,shareef_abdur-rahim,1996.0,2007.0,N,Y,Y,1996,1,3,N
4,76006,Forest,Able,Forest Able,"Able, Forest",F. Able,forest-able,1932-07-27 00:00:00,Western Kentucky,USA,Western Kentucky/USA,6-3,180,1.0,6,Guard,Inactive,N,1610612755,Nationals,PHI,sixers,Syracuse,HISTADD_frosty_able,1956.0,1956.0,N,Y,Y,1956,None,None,N


In [11]:
# List all tables using list_tables function
tables = list_tables()
print("Tables in the database:")
print(tables)

Tables in the database:
['game', 'game_summary', 'other_stats', 'officials', 'inactive_players', 'game_info', 'line_score', 'play_by_play', 'player', 'team', 'common_player_info', 'team_details', 'team_history', 'draft_combine_stats', 'draft_history', 'team_info_common']


Will be exploring each table one by one

### Game Table

In [117]:
# Defining table name
table = 'game' # Update for desired table

In [13]:
# Print data table shape
shape = get_table_shape(table)
print(f"The table '{table}' has {shape[0]} rows and {shape[1]} columns.")

The table 'game' has 65698 rows and 55 columns.


In [14]:
# List all columns in table using list_columns function
columns = list_columns(table)
print(f'Columns in {table} table')
print(columns)

Columns in game table
['season_id', 'team_id_home', 'team_abbreviation_home', 'team_name_home', 'game_id', 'game_date', 'matchup_home', 'wl_home', 'min', 'fgm_home', 'fga_home', 'fg_pct_home', 'fg3m_home', 'fg3a_home', 'fg3_pct_home', 'ftm_home', 'fta_home', 'ft_pct_home', 'oreb_home', 'dreb_home', 'reb_home', 'ast_home', 'stl_home', 'blk_home', 'tov_home', 'pf_home', 'pts_home', 'plus_minus_home', 'video_available_home', 'team_id_away', 'team_abbreviation_away', 'team_name_away', 'matchup_away', 'wl_away', 'fgm_away', 'fga_away', 'fg_pct_away', 'fg3m_away', 'fg3a_away', 'fg3_pct_away', 'ftm_away', 'fta_away', 'ft_pct_away', 'oreb_away', 'dreb_away', 'reb_away', 'ast_away', 'stl_away', 'blk_away', 'tov_away', 'pf_away', 'pts_away', 'plus_minus_away', 'video_available_away', 'season_type']


In [15]:
# Data frame that shows only columns with null values and their count
check_null_values(table)

,wl_home,fgm_home,fga_home,fg_pct_home,fg3m_home,fg3a_home,fg3_pct_home,ftm_home,fta_home,ft_pct_home,oreb_home,dreb_home,reb_home,ast_home,stl_home,blk_home,tov_home,pf_home,wl_away,fgm_away,fga_away,fg_pct_away,fg3m_away,fg3a_away,fg3_pct_away,ftm_away,fta_away,ft_pct_away,oreb_away,dreb_away,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away
0,2,13,15447,15490,13218,18683,19074,16,3004,3009,18936,18999,15729,15805,18849,18626,18684,2856,2,13,15447,15489,13218,18683,18962,13,3004,3006,18936,18998,15725,15801,18849,18625,18685,2851


In [16]:
# Count duplicate rows in table
count_duplicate_entries(table)

0

In [122]:
# Print date range for this table using get_date_range function
date_column = 'game_date'
date_range = get_date_range(table, date_column)
print('Oldest date of recorded games: ', date_range[0])
print('Most recent date of recorded games: ', date_range[1])

Oldest date of recorded games:  1946-11-01 00:00:00
Most recent date of recorded games:  2023-06-12 00:00:00


Data Notes:
- Unique Season Ids are assigned to group annual pre-season, regular-season, playoff, and all-star games. 
- Shorted 2011 Pre-Season and Regular Season due to [lockout](https://en.wikipedia.org/wiki/2011–12_NBA_season)
- Missing 2012 Regular season games

### Game Summary Table

In [126]:
# Defining table name
table = 'game_summary' # Update for desired table

In [18]:
# Print data table shape
shape = get_table_shape(table)
print(f"The table '{table}' has {shape[0]} rows and {shape[1]} columns.")

The table 'game_summary' has 58110 rows and 14 columns.


In [19]:
# List all columns in table using list_columns function
columns = list_columns( table)
print(f'Columns in {table} table')
print(columns)

Columns in game_summary table
['game_date_est', 'game_sequence', 'game_id', 'game_status_id', 'game_status_text', 'gamecode', 'home_team_id', 'visitor_team_id', 'season', 'live_period', 'live_pc_time', 'natl_tv_broadcaster_abbreviation', 'live_period_time_bcast', 'wh_status']


In [20]:
# Data frame that shows only columns with null values and their count
check_null_values(table)

,game_sequence,live_pc_time,natl_tv_broadcaster_abbreviation
0,25532,25986,51907


In [21]:
# Count duplicate rows in table
count_duplicate_entries(table)

40

In [22]:
# Run to view duplicate entries
#get_duplicate_rows(table)

In [127]:
# Print date range for this table using get_date_range function
date_column = 'game_date_est'
date_range = get_date_range(table, date_column)
print('Oldest date of recorded games: ', date_range[0])
print('Most recent date of recorded games: ', date_range[1])

Oldest date of recorded games:  1946-11-01 00:00:00
Most recent date of recorded games:  2023-06-12 00:00:00


### Other Stats Table

In [23]:
# Defining table name
table = 'other_stats' # Update for desired table

In [24]:
# Print data table shape
shape = get_table_shape(table)
print(f"The table '{table}' has {shape[0]} rows and {shape[1]} columns.")

The table 'other_stats' has 28271 rows and 26 columns.


In [25]:
# List all columns in table using list_columns function
columns = list_columns(table)
print(f'Columns in {table} table')
print(columns)

Columns in other_stats table
['game_id', 'league_id', 'team_id_home', 'team_abbreviation_home', 'team_city_home', 'pts_paint_home', 'pts_2nd_chance_home', 'pts_fb_home', 'largest_lead_home', 'lead_changes', 'times_tied', 'team_turnovers_home', 'total_turnovers_home', 'team_rebounds_home', 'pts_off_to_home', 'team_id_away', 'team_abbreviation_away', 'team_city_away', 'pts_paint_away', 'pts_2nd_chance_away', 'pts_fb_away', 'largest_lead_away', 'team_turnovers_away', 'total_turnovers_away', 'team_rebounds_away', 'pts_off_to_away']


In [26]:
# Data frame that shows only columns with null values and their count
check_null_values(table)

,team_turnovers_home,total_turnovers_home,team_rebounds_home,pts_off_to_home,team_turnovers_away,total_turnovers_away,team_rebounds_away,pts_off_to_away
0,2,316,1998,2123,2,316,1998,2123


In [27]:
# Count duplicate rows in table
count_duplicate_entries(table)

10

In [28]:
# Run to view duplicate entries
#get_duplicate_rows(table)

### Officials Table

In [29]:
# Defining table name
table = 'officials' # Update for desired table

In [30]:
# Print data table shape
shape = get_table_shape(table)
print(f"The table '{table}' has {shape[0]} rows and {shape[1]} columns.")

The table 'officials' has 70971 rows and 5 columns.


In [31]:
# List all columns in table using list_columns function
columns = list_columns(table)
print(f'Columns in {table} table')
print(columns)

Columns in officials table
['game_id', 'official_id', 'first_name', 'last_name', 'jersey_num']


In [32]:
# Data frame that shows only columns with null values and their count
check_null_values(table)

,jersey_num
0,183


In [33]:
# Count duplicate rows in table
count_duplicate_entries(table)

30

In [34]:
# Run to view duplicate entries
#get_duplicate_rows(table)

### Inactive Players Table

In [35]:
# Defining table name
table = 'inactive_players' # Update for desired table

In [36]:
# Print data table shape
shape = get_table_shape(table)
print(f"The table '{table}' has {shape[0]} rows and {shape[1]} columns.")

The table 'inactive_players' has 110191 rows and 9 columns.


In [37]:
# List all columns in table using list_columns function
columns = list_columns(table)
print(f'Columns in {table} table')
print(columns)

Columns in inactive_players table
['game_id', 'player_id', 'first_name', 'last_name', 'jersey_num', 'team_id', 'team_city', 'team_name', 'team_abbreviation']


In [38]:
# Data frame that shows only columns with null values and their count
check_null_values(table)

""
0


In [39]:
# Count duplicate rows in table
count_duplicate_entries(table)

7

In [40]:
# Run to view duplicate entries
#get_duplicate_rows(table)

### Game Info Table

In [124]:
# Defining table name
table = 'game_info' # Update for desired table

In [42]:
# Print data table shape
shape = get_table_shape(table)
print(f"The table '{table}' has {shape[0]} rows and {shape[1]} columns.")

The table 'game_info' has 58053 rows and 4 columns.


In [43]:
# List all columns in table using list_columns function
columns = list_columns(table)
print(f'Columns in {table} table')
print(columns)

Columns in game_info table
['game_id', 'game_date', 'attendance', 'game_time']


In [44]:
# Data frame that shows only columns with null values and their count
check_null_values(table)

,attendance
0,5380


In [45]:
# game_time has several empty cells not counted as nulls (28111)

In [46]:
# Count duplicate rows in table
count_duplicate_entries(table)

40

In [47]:
# Run to view duplicate entries
#get_duplicate_rows(table)

In [125]:
# Print date range for this table using get_date_range function
date_column = 'game_date'
date_range = get_date_range(table, date_column)
print('Oldest date of recorded games: ', date_range[0])
print('Most recent date of recorded games: ', date_range[1])

Oldest date of recorded games:  1946-11-01 00:00:00
Most recent date of recorded games:  2023-06-12 00:00:00


Data Notes:
- Attendance figures for 2020 and 2021 season affected due to COVID

### Line Score Table

In [48]:
# Defining table name
table = 'line_score' # Update for desired table

In [49]:
# Print data table shape
shape = get_table_shape(table)
print(f"The table '{table}' has {shape[0]} rows and {shape[1]} columns.")

The table 'line_score' has 58053 rows and 43 columns.


In [50]:
# List all columns in table using list_columns function
columns = list_columns(table)
print(f'Columns in {table} table')
print(columns)

Columns in line_score table
['game_date_est', 'game_sequence', 'game_id', 'team_id_home', 'team_abbreviation_home', 'team_city_name_home', 'team_nickname_home', 'team_wins_losses_home', 'pts_qtr1_home', 'pts_qtr2_home', 'pts_qtr3_home', 'pts_qtr4_home', 'pts_ot1_home', 'pts_ot2_home', 'pts_ot3_home', 'pts_ot4_home', 'pts_ot5_home', 'pts_ot6_home', 'pts_ot7_home', 'pts_ot8_home', 'pts_ot9_home', 'pts_ot10_home', 'pts_home', 'team_id_away', 'team_abbreviation_away', 'team_city_name_away', 'team_nickname_away', 'team_wins_losses_away', 'pts_qtr1_away', 'pts_qtr2_away', 'pts_qtr3_away', 'pts_qtr4_away', 'pts_ot1_away', 'pts_ot2_away', 'pts_ot3_away', 'pts_ot4_away', 'pts_ot5_away', 'pts_ot6_away', 'pts_ot7_away', 'pts_ot8_away', 'pts_ot9_away', 'pts_ot10_away', 'pts_away']


In [51]:
# Data frame that shows only columns with null values and their count
check_null_values(table)

,game_sequence,pts_qtr1_home,pts_qtr2_home,pts_qtr3_home,pts_qtr4_home,pts_ot1_home,pts_ot2_home,pts_ot3_home,pts_ot4_home,pts_ot5_home,pts_ot6_home,pts_ot7_home,pts_ot8_home,pts_ot9_home,pts_ot10_home,pts_qtr1_away,pts_qtr2_away,pts_qtr3_away,pts_qtr4_away,pts_ot1_away,pts_ot2_away,pts_ot3_away,pts_ot4_away,pts_ot5_away,pts_ot6_away,pts_ot7_away,pts_ot8_away,pts_ot9_away,pts_ot10_away
0,25532,1004,1013,1045,1044,25759,27051,27243,27270,45577,45578,45578,45578,45578,45578,1010,1013,1046,1046,25759,27051,27243,27270,45577,45578,45578,45578,45578,45578


In [52]:
# Count duplicate rows in table
count_duplicate_entries(table)

40

In [53]:
# Run to view duplicate entries
#get_duplicate_rows(table)

### Play by Play Table

In [54]:
# Defining table name
table = 'play_by_play' # Update for desired table

In [55]:
# Print data table shape
shape = get_table_shape(table)
print(f"The table '{table}' has {shape[0]} rows and {shape[1]} columns.")

The table 'play_by_play' has 13592899 rows and 34 columns.


In [56]:
# List all columns in table using list_columns function
columns = list_columns(table)
print(f'Columns in {table} table')
print(columns)

Columns in play_by_play table
['game_id', 'eventnum', 'eventmsgtype', 'eventmsgactiontype', 'period', 'wctimestring', 'pctimestring', 'homedescription', 'neutraldescription', 'visitordescription', 'score', 'scoremargin', 'person1type', 'player1_id', 'player1_name', 'player1_team_id', 'player1_team_city', 'player1_team_nickname', 'player1_team_abbreviation', 'person2type', 'player2_id', 'player2_name', 'player2_team_id', 'player2_team_city', 'player2_team_nickname', 'player2_team_abbreviation', 'person3type', 'player3_id', 'player3_name', 'player3_team_id', 'player3_team_city', 'player3_team_nickname', 'player3_team_abbreviation', 'video_available_flag']


In [57]:
# Data frame that shows only columns with null values and their count
check_null_values(table)

,homedescription,neutraldescription,visitordescription,score,scoremargin,person1type,player1_name,player1_team_id,player1_team_city,player1_team_nickname,player1_team_abbreviation,player2_name,player2_team_id,player2_team_city,player2_team_nickname,player2_team_abbreviation,player3_name,player3_team_id,player3_team_city,player3_team_nickname,player3_team_abbreviation
0,6529440,13273291,6634518,10028436,10028436,3298,1208875,1215858,1215858,1215858,1215858,9683745,9660454,9660454,9660454,9660454,13251785,13246931,13246931,13246931,13246931


In [58]:
# Count duplicate rows in table
count_duplicate_entries(table)

7360

In [59]:
# Run to view duplicate entries
#get_duplicate_rows(table).head()

### Player Table

In [60]:
# Defining table name
table = 'player' # Update for desired table

In [61]:
# Print data table shape
shape = get_table_shape(table)
print(f"The table '{table}' has {shape[0]} rows and {shape[1]} columns.")

The table 'player' has 4815 rows and 5 columns.


In [62]:
# List all columns in table using list_columns function
columns = list_columns(table)
print(f'Columns in {table} table')
print(columns)

Columns in player table
['id', 'full_name', 'first_name', 'last_name', 'is_active']


In [63]:
# Data frame that shows only columns with null values and their count
check_null_values(table)

""
0


In [64]:
# Count duplicate rows in table
count_duplicate_entries(table)

0

### Team Table

In [65]:
# Defining table name
table = 'team' # Update for desired table

In [66]:
# Print data table shape
shape = get_table_shape(table)
print(f"The table '{table}' has {shape[0]} rows and {shape[1]} columns.")

The table 'team' has 30 rows and 7 columns.


In [67]:
# List all columns in table using list_columns function
columns = list_columns(table)
print(f'Columns in {table} table')
print(columns)

Columns in team table
['id', 'full_name', 'abbreviation', 'nickname', 'city', 'state', 'year_founded']


In [68]:
# Data frame that shows only columns with null values and their count
check_null_values(table)

""
0


In [69]:
# Count duplicate rows in table
count_duplicate_entries(table)

0

### Common Player Info Table

In [70]:
# Defining table name
table = 'common_player_info' # Update for desired table

In [71]:
# Print data table shape
shape = get_table_shape(table)
print(f"The table '{table}' has {shape[0]} rows and {shape[1]} columns.")

The table 'common_player_info' has 3632 rows and 33 columns.


In [72]:
# List all columns in table using list_columns function
columns = list_columns(table)
print(f'Columns in {table} table')
print(columns)

Columns in common_player_info table
['person_id', 'first_name', 'last_name', 'display_first_last', 'display_last_comma_first', 'display_fi_last', 'player_slug', 'birthdate', 'school', 'country', 'last_affiliation', 'height', 'weight', 'season_exp', 'jersey', 'position', 'rosterstatus', 'games_played_current_season_flag', 'team_id', 'team_name', 'team_abbreviation', 'team_code', 'team_city', 'playercode', 'from_year', 'to_year', 'dleague_flag', 'nba_flag', 'games_played_flag', 'draft_year', 'draft_round', 'draft_number', 'greatest_75_flag']


In [73]:
# Data frame that shows only columns with null values and their count
check_null_values(table)

,school,country,playercode,draft_round,draft_number
0,1,1,1,146,198


In [74]:
# Count duplicate rows in table
count_duplicate_entries(table)

0

### Team Details Table

In [75]:
# Defining table name
table = 'team_details' # Update for desired table

In [76]:
# Print data table shape
shape = get_table_shape(table)
print(f"The table '{table}' has {shape[0]} rows and {shape[1]} columns.")

The table 'team_details' has 27 rows and 14 columns.


In [77]:
# List all columns in table using list_columns function
columns = list_columns(table)
print(f'Columns in {table} table')
print(columns)

Columns in team_details table
['team_id', 'abbreviation', 'nickname', 'yearfounded', 'city', 'arena', 'arenacapacity', 'owner', 'generalmanager', 'headcoach', 'dleagueaffiliation', 'facebook', 'instagram', 'twitter']


In [78]:
# Data frame that shows only columns with null values and their count
check_null_values(table)

,arenacapacity
0,10


In [79]:
# Count duplicate rows in table
count_duplicate_entries(table)

0

Data Notes:
- Table is missing three teams, that I will impute later
- Arena Capacity are missing values, that I will impute as well

### Team History Table

In [80]:
# Defining table name
table = 'team_history' # Update for desired table

In [81]:
# Print data table shape
shape = get_table_shape(table)
print(f"The table '{table}' has {shape[0]} rows and {shape[1]} columns.")

The table 'team_history' has 50 rows and 5 columns.


In [82]:
# List all columns in table using list_columns function
columns = list_columns(table)
print(f'Columns in {table} table')
print(columns)

Columns in team_history table
['team_id', 'city', 'nickname', 'year_founded', 'year_active_till']


In [83]:
# Data frame that shows only columns with null values and their count
check_null_values(table)

""
0


In [84]:
# Count duplicate rows in table
count_duplicate_entries(table)

0

### Draft Combine Stats Table

In [85]:
# Defining table name
table = 'draft_combine_stats' # Update for desired table

In [86]:
# Print data table shape
shape = get_table_shape(table)
print(f"The table '{table}' has {shape[0]} rows and {shape[1]} columns.")

The table 'draft_combine_stats' has 1633 rows and 47 columns.


In [87]:
# List all columns in table using list_columns function
columns = list_columns(table)
print(f'Columns in {table} table')
print(columns)

Columns in draft_combine_stats table
['season', 'player_id', 'first_name', 'last_name', 'player_name', 'position', 'height_wo_shoes', 'height_wo_shoes_ft_in', 'height_w_shoes', 'height_w_shoes_ft_in', 'weight', 'wingspan', 'wingspan_ft_in', 'standing_reach', 'standing_reach_ft_in', 'body_fat_pct', 'hand_length', 'hand_width', 'standing_vertical_leap', 'max_vertical_leap', 'lane_agility_time', 'modified_lane_agility_time', 'three_quarter_sprint', 'bench_press', 'spot_fifteen_corner_left', 'spot_fifteen_break_left', 'spot_fifteen_top_key', 'spot_fifteen_break_right', 'spot_fifteen_corner_right', 'spot_college_corner_left', 'spot_college_break_left', 'spot_college_top_key', 'spot_college_break_right', 'spot_college_corner_right', 'spot_nba_corner_left', 'spot_nba_break_left', 'spot_nba_top_key', 'spot_nba_break_right', 'spot_nba_corner_right', 'off_drib_fifteen_break_left', 'off_drib_fifteen_top_key', 'off_drib_fifteen_break_right', 'off_drib_college_break_left', 'off_drib_college_top_key

In [88]:
# Data frame that shows only columns with null values and their count
check_null_values(table)

,height_wo_shoes,height_wo_shoes_ft_in,height_w_shoes,height_w_shoes_ft_in,weight,wingspan,wingspan_ft_in,standing_reach,standing_reach_ft_in,body_fat_pct,hand_length,hand_width,standing_vertical_leap,max_vertical_leap,lane_agility_time,modified_lane_agility_time,three_quarter_sprint,bench_press,spot_fifteen_corner_left,spot_fifteen_break_left,spot_fifteen_top_key,spot_fifteen_break_right,spot_fifteen_corner_right,spot_college_corner_left,spot_college_break_left,spot_college_top_key,spot_college_break_right,spot_college_corner_right,spot_nba_corner_left,spot_nba_break_left,spot_nba_top_key,spot_nba_break_right,spot_nba_corner_right,off_drib_fifteen_break_left,off_drib_fifteen_top_key,off_drib_fifteen_break_right,off_drib_college_break_left,off_drib_college_top_key,off_drib_college_break_right,on_move_fifteen,on_move_college
0,58,58,428,428,59,58,58,59,59,357,794,794,237,237,246,1124,246,538,1529,1527,1527,1527,1527,1341,1462,1462,1462,1462,1381,1381,1381,1381,1381,1432,1432,1432,1483,1602,1602,1449,1465


In [89]:
# Count duplicate rows in table
count_duplicate_entries(table)

0

### Draft History Table

In [90]:
# Defining table name
table = 'draft_history' # Update for desired table

In [91]:
# Print data table shape
shape = get_table_shape(table)
print(f"The table '{table}' has {shape[0]} rows and {shape[1]} columns.")

The table 'draft_history' has 8257 rows and 14 columns.


In [92]:
# List all columns in table using list_columns function
columns = list_columns(table)
print(f'Columns in {table} table')
print(columns)

Columns in draft_history table
['person_id', 'player_name', 'season', 'round_number', 'round_pick', 'overall_pick', 'draft_type', 'team_id', 'team_city', 'team_name', 'team_abbreviation', 'organization', 'organization_type', 'player_profile_flag']


In [93]:
# Data frame that shows only columns with null values and their count
check_null_values(table)

""
0


In [94]:
# Count duplicate rows in table
count_duplicate_entries(table)

0

### Team Info Common Table

In [95]:
# Defining table name
table = 'team_info_common' # Update for desired table

In [96]:
# Print data table shape
shape = get_table_shape(table)
print(f"The table '{table}' has {shape[0]} rows and {shape[1]} columns.")

The table 'team_info_common' has 0 rows and 26 columns.


In [97]:
# List all columns in table using list_columns function
columns = list_columns(table)
print(f'Columns in {table} table')
print(columns)

Columns in team_info_common table
['team_id', 'season_year', 'team_city', 'team_name', 'team_abbreviation', 'team_conference', 'team_division', 'team_code', 'team_slug', 'w', 'l', 'pct', 'conf_rank', 'div_rank', 'min_year', 'max_year', 'league_id', 'season_id', 'pts_rank', 'pts_pg', 'reb_rank', 'reb_pg', 'ast_rank', 'ast_pg', 'opp_pts_rank', 'opp_pts_pg']


In [98]:
# Data frame that shows only columns with null values and their count
check_null_values(table)

,team_id,season_year,team_city,team_name,team_abbreviation,team_conference,team_division,team_code,team_slug,w,l,pct,conf_rank,div_rank,min_year,max_year,league_id,season_id,pts_rank,pts_pg,reb_rank,reb_pg,ast_rank,ast_pg,opp_pts_rank,opp_pts_pg
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [99]:
# Count duplicate rows in table
count_duplicate_entries(table)

0

Data Notes:
- Unfortunately table was empty, no data available from Kaggle data set

## Loading Select SQLite Data

In [107]:
# Selecting features to load
game_columns = 'game_id, team_id_home, team_id_away, season_id, season_type, wl_home, wl_away, pts_home, pts_away, plus_minus_home, plus_minus_away, min'
game_info_columns = 'game_id, game_date, attendance'
game_sum_columns = 'game_id, season'

team_columns = 'id AS team_id, full_name, abbreviation, city, state, year_founded'
team_details_columns = 'team_id, arena, arenacapacity'

In [108]:
# Loading data from tables
game_df = pd.read_sql_query(f"SELECT {game_columns} FROM game", conn)
game_info_df = pd.read_sql_query(f"SELECT {game_info_columns} FROM game_info", conn)
game_summary_df = pd.read_sql_query(f"SELECT {game_sum_columns} FROM game_summary", conn)

team_df = pd.read_sql_query(f"SELECT {team_columns} FROM team", conn)
team_details_df = pd.read_sql_query(f"SELECT {team_details_columns} FROM team_details", conn)


In [109]:
# Close the connection to SQLite data base
close_connection()

Database connection closed.


Merging data into two data frames, one for games and one for teams

In [112]:
print(game_df.shape)
print(game_info_df.shape)
print(game_summary_df.shape)

(65698, 12)
(58053, 3)
(58110, 2)


In [128]:
# Merge the game DataFrames on 'game_id'
game_df = pd.merge(game_df, game_info_df, on='game_id', how='inner')
game_df = pd.merge(game_df, game_summary_df, on='game_id', how='inner')
game_df.shape

(58359, 15)

In [129]:
# Merge the team DataFrames on 'team_id'
team_df = pd.merge(team_df, team_details_df, on='team_id', how='inner')
team_df.shape

(27, 8)

## Transforming Data

In [130]:
game_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58359 entries, 0 to 58358
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   game_id          58359 non-null  object 
 1   team_id_home     58359 non-null  object 
 2   team_id_away     58359 non-null  object 
 3   season_id        58359 non-null  object 
 4   season_type      58359 non-null  object 
 5   wl_home          58357 non-null  object 
 6   wl_away          58357 non-null  object 
 7   pts_home         58359 non-null  float64
 8   pts_away         58359 non-null  float64
 9   plus_minus_home  58359 non-null  int64  
 10  plus_minus_away  58359 non-null  int64  
 11  min              58359 non-null  int64  
 12  game_date        58359 non-null  object 
 13  attendance       52785 non-null  float64
 14  season           58359 non-null  object 
dtypes: float64(3), int64(3), object(9)
memory usage: 6.7+ MB


In [131]:
team_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   team_id        27 non-null     object 
 1   full_name      27 non-null     object 
 2   abbreviation   27 non-null     object 
 3   city           27 non-null     object 
 4   state          27 non-null     object 
 5   year_founded   27 non-null     float64
 6   arena          27 non-null     object 
 7   arenacapacity  17 non-null     float64
dtypes: float64(2), object(6)
memory usage: 1.8+ KB


### Type Conversions 

In [132]:
# game_df column conversions
# Change columns game_id, team_id_home, team_id_away, season_id values to a numeric type
game_df['game_id'] = pd.to_numeric(game_df['game_id'])
game_df['team_id_home'] = pd.to_numeric(game_df['team_id_home'])
game_df['team_id_away'] = pd.to_numeric(game_df['team_id_away'])
game_df['season_id'] = pd.to_numeric(game_df['season_id'])
game_df['season'] = pd.to_numeric(game_df['season'])
# Change column game_date values to a datetime
game_df['game_date'] = pd.to_datetime(game_df['game_date'])

In [133]:
# Change team column team_id values to a numeric type
team_df['team_id'] = pd.to_numeric(team_df['team_id'])
team_df['year_founded'] = team_df['year_founded'].astype(int)

### Imputing Data
- Three NBA team were missing from team details table
- Attendance numbers were missing from team details table

In [137]:
# Data sourced from Wikipedia 
# Create a DataFrame for the missing teams 
missing_teams = pd.DataFrame({
    'team_id': [1610612759, 1610612764, 1610612766],
    'full_name': ['San Antonio Spurs', 'Washington Wizards', 'Charlotte Hornets'],
    'abbreviation': ['SAS', 'WAS', 'CHA'],
    'city': ['San Antonio', 'Washington', 'Charlotte'],
    'state': ['Texas', 'District of Columbia', 'North Carolina'],
    'year_founded': [1976, 1961, 1988],
    'arena': ['AT&T Center', 'Capital One Arena', 'Spectrum Center'],
    'arenacapacity': [18418, 20356, 19077]
})

# Concatenate the missing teams to the team_df
team_df = pd.concat([team_df, missing_teams], ignore_index=True)

# Verify the updated DataFrame
team_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   team_id        30 non-null     int64  
 1   full_name      30 non-null     object 
 2   abbreviation   30 non-null     object 
 3   city           30 non-null     object 
 4   state          30 non-null     object 
 5   year_founded   30 non-null     int64  
 6   arena          30 non-null     object 
 7   arenacapacity  20 non-null     float64
dtypes: float64(1), int64(2), object(5)
memory usage: 2.0+ KB


Arena Capacity
Data Source: https://en.wikipedia.org/wiki/List_of_NBA_arenas

In [138]:
# Dictionary mapping the specific arena capacity values
arena_capacity_updates = {
    'Smoothie King Center': 16867,
    'Ball Arena': 19520,
    'Chase Center': 18064,
    'Footprint Center': 18422,
    'Moda Center': 19393,
    'Paycom Center': 18203,
    'Scotiabank Arena': 19800,
    'Delta Center': 18300,
    'Wells Fargo Center': 20478,
    'Barclays Center': 17732,
    'Amway Center': 18846,
    'Little Caesars Arena': 20332
}

# Filling in the specific arena capacities based on the arena names
team_df['arenacapacity'] = team_df.apply(
    lambda row: arena_capacity_updates.get(row['arena'], row['arenacapacity']),
    axis=1
)

# Verify the DataFrame after filling NaN values
team_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   team_id        30 non-null     int64  
 1   full_name      30 non-null     object 
 2   abbreviation   30 non-null     object 
 3   city           30 non-null     object 
 4   state          30 non-null     object 
 5   year_founded   30 non-null     int64  
 6   arena          30 non-null     object 
 7   arenacapacity  30 non-null     float64
dtypes: float64(1), int64(2), object(5)
memory usage: 2.0+ KB


### Cleaning Data

In [139]:
game_df.shape

(58359, 15)

Removing Duplicates

In [140]:
duplicates = game_df[game_df.duplicated()]
duplicates.shape

(290, 15)

In [141]:
game_df.drop_duplicates(inplace=True)

In [142]:
duplicates = team_df[team_df.duplicated()]
duplicates.shape

(0, 8)

Removing All-Star Games

In [143]:
game_df = game_df.drop(game_df[(game_df.season_type == 'All-Star') | (game_df.season_type == 'All Star')].index)

Dropping games without a matching team in team_df

In [144]:
# Extracting the relevant team_id column from the team_df
team_ids = team_df['team_id'].unique()

In [146]:
# Filtering the game_df to drop rows where either home or away team_id is not in the team_df
game_df = game_df[
    (game_df['team_id_home'].isin(team_ids)) &
    (game_df['team_id_away'].isin(team_ids))
]

Handling Missing Values 

In [148]:
# Dropping games with no recorded win 
game_df = game_df.drop(game_df[(game_df.wl_home.isnull())].index)

In [149]:
game_df.shape

(56362, 15)

### Feature Engineering
- Creating new season stats table to keep track of team performance for each season
- Categorizing game minutes played to identify games that go into overtime
- Categorizing games played in different months and days of the week
- Identifying team city location coordinates 

##### Season Stats Table

In [152]:
# Consolidate team_id_home and team_id_away columns into a single 'team_id' column
# We will consider both home and away team IDs by stacking the data

# Combine home wins/losses
home_stats = game_df.groupby(['season_id', 'team_id_home']).agg(wins_at_home=('wl_home', lambda x: (x == 'W').sum()), 
                                                                losses_at_home=('wl_home', lambda x: (x == 'L').sum())).reset_index()
home_stats = home_stats.rename(columns={'team_id_home': 'team_id'})


# Combine away wins/losses
away_stats = game_df.groupby(['season_id', 'team_id_away']).agg(away_wins=('wl_away', lambda x: (x == 'W').sum()), 
                                                                away_losses=('wl_away', lambda x: (x == 'L').sum())).reset_index()
away_stats = away_stats.rename(columns={'team_id_away': 'team_id'})

In [153]:
# Merge the home and away stats
season_stats = pd.merge(home_stats, away_stats, on=['season_id', 'team_id'], how='outer')

In [154]:
# Fill NaN values with 0
season_stats = season_stats.fillna(0)

In [155]:
# Calculate total wins and losses
season_stats['season_w'] = season_stats['wins_at_home'] + season_stats['away_wins']
season_stats['season_l'] = season_stats['losses_at_home'] + season_stats['away_losses']

In [156]:
# Reorder columns
season_stats = season_stats[['season_id', 'team_id', 'season_w', 'season_l', 'wins_at_home', 'losses_at_home', 'away_wins', 'away_losses']]

Points per game

In [157]:
# Calculate total points scored at home and away per season
total_pts_home = game_df.groupby(['season_id', 'team_id_home'])['pts_home'].sum().reset_index()
total_pts_away = game_df.groupby(['season_id', 'team_id_away'])['pts_away'].sum().reset_index()

# Calculate the total games played at home and away per season
total_games_home = game_df.groupby(['season_id', 'team_id_home'])['game_id'].count().reset_index()
total_games_away = game_df.groupby(['season_id', 'team_id_away'])['game_id'].count().reset_index()

In [158]:
# Rename Columns
total_pts_home = total_pts_home.rename(columns={'team_id_home': 'team_id', 'pts_home': 'total_pts_at_home'})
total_pts_away = total_pts_away.rename(columns={'team_id_away': 'team_id', 'pts_away': 'total_away_pts'})
total_games_home = total_games_home.rename(columns={'team_id_home': 'team_id', 'game_id': 'games_at_home'})
total_games_away = total_games_away.rename(columns={'team_id_away': 'team_id', 'game_id': 'away_games'})

In [159]:
# Merge home and away points and games into season_stats
season_stats = season_stats.merge(total_pts_home, on=['season_id', 'team_id'], how='left')
season_stats = season_stats.merge(total_pts_away, on=['season_id', 'team_id'], how='left')
season_stats = season_stats.merge(total_games_home, on=['season_id', 'team_id'], how='left')
season_stats = season_stats.merge(total_games_away, on=['season_id', 'team_id'], how='left')

In [160]:
# Calculate total recorded games played for each team in each season
season_stats['games_this_season'] = season_stats['games_at_home'] + season_stats['away_games']

In [161]:
# Fill any NaN values with 0 (in case a team has no home/away games in the dataset)
season_stats = season_stats.fillna(0)

In [162]:
# Calculate average points per game at home using apply
season_stats['pts_pg_at_home'] = season_stats.apply(
    lambda row: round(row['total_pts_at_home'] / row['games_at_home'], 1) if row['games_at_home'] != 0 else 0, axis=1)

# Calculate average points per game away using apply
season_stats['away_pts_pg'] = season_stats.apply(
    lambda row: round(row['total_away_pts'] / row['away_games'], 1) if row['away_games'] != 0 else 0, axis=1)

In [163]:
# Calculate overall points per game per season
season_stats['season_pts_pg'] = round((season_stats['total_pts_at_home'] + season_stats['total_away_pts']) / \
                            (season_stats['games_at_home'] + season_stats['away_games']), 1)

# Display the first few rows to verify
season_stats[['season_id', 'team_id', 'pts_pg_at_home', 'away_pts_pg', 'season_pts_pg']].head()

,season_id,team_id,pts_pg_at_home,away_pts_pg,season_pts_pg
0,12005,1610612737,98.5,90.3,93.6
1,12005,1610612738,95.0,95.5,95.2
2,12005,1610612739,90.3,102.8,97.4
3,12005,1610612740,91.5,91.0,91.2
4,12005,1610612741,103.7,90.7,97.2


Win percentage

In [164]:
# Calculate overall winning percentage
season_stats['season_win_pct'] = round(season_stats['season_w'] / (season_stats['season_w'] + season_stats['season_l']), 3)

# Calculate home winning percentage using apply
season_stats['season_win_pct_at_home'] = season_stats.apply(
    lambda row: 0 if row['games_at_home'] == 0 else round(row['wins_at_home'] / (row['wins_at_home'] + row['losses_at_home']), 3), axis=1)

# Calculate away winning percentage using apply
season_stats['season_away_win_pct'] = season_stats.apply(
    lambda row: 0 if row['away_games'] == 0 else round(row['away_wins'] / (row['away_wins'] + row['away_losses']), 3), axis=1)

In [165]:
season_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2766 entries, 0 to 2765
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   season_id               2766 non-null   int64  
 1   team_id                 2766 non-null   int64  
 2   season_w                2766 non-null   float64
 3   season_l                2766 non-null   float64
 4   wins_at_home            2766 non-null   float64
 5   losses_at_home          2766 non-null   float64
 6   away_wins               2766 non-null   float64
 7   away_losses             2766 non-null   float64
 8   total_pts_at_home       2766 non-null   float64
 9   total_away_pts          2766 non-null   float64
 10  games_at_home           2766 non-null   float64
 11  away_games              2766 non-null   float64
 12  games_this_season       2766 non-null   float64
 13  pts_pg_at_home          2766 non-null   float64
 14  away_pts_pg             2766 non-null   

Average Game Attendance

In [166]:
# Combine home wins/losses and aggregate attendance
average_attendance = game_df.groupby(['season_id', 'team_id_home']).agg(avg_attendance=('attendance', 'mean')).reset_index()
average_attendance = average_attendance.rename(columns={'team_id_home': 'team_id'})

# round the attendance calculation
average_attendance['avg_attendance'] = average_attendance['avg_attendance'].round()

In [167]:
average_attendance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2746 entries, 0 to 2745
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   season_id       2746 non-null   int64  
 1   team_id         2746 non-null   int64  
 2   avg_attendance  2543 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 64.5 KB


In [169]:
# Merge the home and away stats
season_stats = pd.merge(season_stats, average_attendance, on=['season_id', 'team_id'], how='outer')

In [171]:
season_stats.columns

Index(['season_id', 'team_id', 'season_w', 'season_l', 'wins_at_home', 'losses_at_home', 'away_wins', 'away_losses', 'total_pts_at_home', 'total_away_pts', 'games_at_home', 'away_games', 'games_this_season', 'pts_pg_at_home', 'away_pts_pg', 'season_pts_pg', 'season_win_pct', 'season_win_pct_at_home', 'season_away_win_pct', 'avg_attendance'], dtype='object')

In [172]:
# Reorder columns
season_stats = season_stats[['season_id', 'team_id', 'avg_attendance',
                             'season_w', 'season_l', 'wins_at_home', 'losses_at_home', 'away_wins', 'away_losses', 
                             'games_this_season', 'games_at_home', 'away_games', 'total_pts_at_home', 'total_away_pts',  
                             'season_pts_pg', 'pts_pg_at_home', 'away_pts_pg',  
                             'season_win_pct', 'season_win_pct_at_home', 'season_away_win_pct']]

In [173]:
season_stats.shape

(2766, 20)

##### Game Duration

In [174]:
# Apply the function to create the new column
game_df['game_duration'] = game_df['min'].apply(game_duration)

In [175]:
game_df['game_duration'].value_counts()

game_duration
Regulation        53344
Overtime           2591
Extra Overtime      425
Ended Early           2
Name: count, dtype: int64

##### Game Day Details

In [176]:
# Adding new columns to game_df
game_df['day_of_week'] = pd.to_datetime(game_df['game_date']).dt.day_name()
game_df['month'] = pd.to_datetime(game_df['game_date']).dt.month
# Verify changes 
game_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 56362 entries, 41 to 58350
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   game_id          56362 non-null  int64         
 1   team_id_home     56362 non-null  int64         
 2   team_id_away     56362 non-null  int64         
 3   season_id        56362 non-null  int64         
 4   season_type      56362 non-null  object        
 5   wl_home          56362 non-null  object        
 6   wl_away          56362 non-null  object        
 7   pts_home         56362 non-null  float64       
 8   pts_away         56362 non-null  float64       
 9   plus_minus_home  56362 non-null  int64         
 10  plus_minus_away  56362 non-null  int64         
 11  min              56362 non-null  int64         
 12  game_date        56362 non-null  datetime64[ns]
 13  attendance       52539 non-null  float64       
 14  season           56362 non-null  int64    

##### Location Info

In [177]:
# Initialize the geolocator
geolocator = Nominatim(user_agent="team_geocoder")


In [178]:
# Apply the function to get coordinates for each city
team_df['latitude'], team_df['longitude'] = zip(*team_df['city'].apply(get_coordinates))
# Verify the DataFrame
team_df

,team_id,full_name,abbreviation,city,state,year_founded,arena,arenacapacity,latitude,longitude
0,1610612737,Atlanta Hawks,ATL,Atlanta,Georgia,1949,State Farm Arena,18729.0,33.748992,-84.390264
1,1610612738,Boston Celtics,BOS,Boston,Massachusetts,1946,TD Garden,18624.0,42.355433,-71.060511
2,1610612739,Cleveland Cavaliers,CLE,Cleveland,Ohio,1970,Rocket Mortgage FieldHouse,20562.0,41.499657,-81.693677
3,1610612740,New Orleans Pelicans,NOP,New Orleans,Louisiana,2002,Smoothie King Center,16867.0,29.975998,-90.078213
4,1610612741,Chicago Bulls,CHI,Chicago,Illinois,1966,United Center,21711.0,41.875562,-87.624421
5,1610612742,Dallas Mavericks,DAL,Dallas,Texas,1980,American Airlines Center,19200.0,32.776272,-96.796856
6,1610612743,Denver Nuggets,DEN,Denver,Colorado,1976,Ball Arena,19520.0,39.739236,-104.984862
7,1610612744,Golden State Warriors,GSW,Golden State,California,1946,Chase Center,18064.0,16.803337,96.178660
8,1610612745,Houston Rockets,HOU,Houston,Texas,1967,Toyota Center,18104.0,29.758938,-95.367697
9,1610612746,Los Angeles Clippers,LAC,Los Angeles,California,1970,Crypto.com Arena,19060.0,34.053691,-118.242766


In [179]:
team_location_df = team_df[['team_id', 'latitude', 'longitude']]
team_location_df.head()

,team_id,latitude,longitude
0,1610612737,33.748992,-84.390264
1,1610612738,42.355433,-71.060511
2,1610612739,41.499657,-81.693677
3,1610612740,29.975998,-90.078213
4,1610612741,41.875562,-87.624421


### Exporting Transformed Data (Optional)

In [180]:
# # Export team location info 
# team_location_df.to_csv('data/team_location.csv', index=False)
# # Export transformed team and game data 
# game_df.to_csv('data/games.csv', index=False)
# team_df.to_csv('data/teams.csv', index=False)
# # Export new season stats data
# season_stats.to_csv('data/season_stats.csv', index=False)

## Merging Data


In [182]:
# Merge to add all home team data
combined_df = pd.merge(game_df, team_df, 
                               left_on='team_id_home', right_on='team_id', suffixes=('', '_home_team'))

# Drop the redundant team_id column from the home merge
combined_df.drop('team_id', axis=1, inplace=True)

# Merge to add all away team data
combined_df = pd.merge(combined_df, team_df, 
                           left_on='team_id_away', right_on='team_id', suffixes=('_home_team', '_away_team'))

# Drop the redundant team_id column from the away merge
combined_df.drop('team_id', axis=1, inplace=True)

# Updated dataframe
combined_df.head()

,game_id,team_id_home,team_id_away,season_id,season_type,wl_home,wl_away,pts_home,pts_away,plus_minus_home,plus_minus_away,min,game_date,attendance,season,game_duration,day_of_week,month,full_name_home_team,abbreviation_home_team,city_home_team,state_home_team,year_founded_home_team,arena_home_team,arenacapacity_home_team,latitude_home_team,longitude_home_team,full_name_away_team,abbreviation_away_team,city_away_team,state_away_team,year_founded_away_team,arena_away_team,arenacapacity_away_team,latitude_away_team,longitude_away_team
0,24600052,1610612744,1610612738,21946,Regular Season,W,L,66.0,54.0,12,-12,0,1946-11-26,NaN,1946,Regulation,Tuesday,11,Golden State Warriors,GSW,Golden State,California,1946,Chase Center,18064.0,16.803337,96.178660,Boston Celtics,BOS,Boston,Massachusetts,1946,TD Garden,18624.0,42.355433,-71.060511
1,24600063,1610612752,1610612744,21946,Regular Season,W,L,64.0,60.0,4,-4,0,1946-11-30,NaN,1946,Regulation,Saturday,11,New York Knicks,NYK,New York,New York,1946,Madison Square Garden,19763.0,40.712728,-74.006015,Golden State Warriors,GSW,Golden State,California,1946,Chase Center,18064.0,16.803337,96.178660
2,24600083,1610612752,1610612738,21946,Regular Season,W,L,62.0,44.0,18,-18,0,1946-12-08,NaN,1946,Regulation,Sunday,12,New York Knicks,NYK,New York,New York,1946,Madison Square Garden,19763.0,40.712728,-74.006015,Boston Celtics,BOS,Boston,Massachusetts,1946,TD Garden,18624.0,42.355433,-71.060511
3,24600095,1610612738,1610612744,21946,Regular Season,W,L,77.0,65.0,12,-12,0,1946-12-14,NaN,1946,Regulation,Saturday,12,Boston Celtics,BOS,Boston,Massachusetts,1946,TD Garden,18624.0,42.355433,-71.060511,Golden State Warriors,GSW,Golden State,California,1946,Chase Center,18064.0,16.803337,96.178660
4,24600119,1610612738,1610612744,21946,Regular Season,L,W,60.0,63.0,-3,3,0,1946-12-27,NaN,1946,Regulation,Friday,12,Boston Celtics,BOS,Boston,Massachusetts,1946,TD Garden,18624.0,42.355433,-71.060511,Golden State Warriors,GSW,Golden State,California,1946,Chase Center,18064.0,16.803337,96.178660


In [183]:
combined_df.shape

(56362, 36)

In [185]:
# Merge to add all home team season stats
combined_df = pd.merge(combined_df, season_stats, 
                       left_on=['team_id_home', 'season_id'], 
                       right_on=['team_id', 'season_id'], 
                       suffixes=('', '_home_team'))

# Drop the redundant team_id column from the home merge
combined_df.drop('team_id', axis=1, inplace=True)

# Merge to add all away team season stats
combined_df = pd.merge(combined_df, season_stats, 
                       left_on=['team_id_away', 'season_id'], 
                       right_on=['team_id', 'season_id'], 
                       suffixes=('_home_team', '_away_team'))

# Drop the redundant team_id column from the away merge
combined_df.drop('team_id', axis=1, inplace=True)
combined_df.head()

,game_id,team_id_home,team_id_away,season_id,season_type,wl_home,wl_away,pts_home,pts_away,plus_minus_home,plus_minus_away,min,game_date,attendance,season,game_duration,day_of_week,month,full_name_home_team,abbreviation_home_team,city_home_team,state_home_team,year_founded_home_team,arena_home_team,arenacapacity_home_team,latitude_home_team,longitude_home_team,full_name_away_team,abbreviation_away_team,city_away_team,state_away_team,year_founded_away_team,arena_away_team,arenacapacity_away_team,latitude_away_team,longitude_away_team,avg_attendance_home_team,season_w_home_team,season_l_home_team,wins_at_home_home_team,losses_at_home_home_team,away_wins_home_team,away_losses_home_team,games_this_season_home_team,games_at_home_home_team,away_games_home_team,total_pts_at_home_home_team,total_away_pts_home_team,season_pts_pg_home_team,pts_pg_at_home_home_team,away_pts_pg_home_team,season_win_pct_home_team,season_win_pct_at_home_home_team,season_away_win_pct_home_team,avg_attendance_away_team,season_w_away_team,season_l_away_team,wins_at_home_away_team,losses_at_home_away_team,away_wins_away_team,away_losses_away_team,games_this_season_away_team,games_at_home_away_team,away_games_away_team,total_pts_at_home_away_team,total_away_pts_away_team,season_pts_pg_away_team,pts_pg_at_home_away_team,away_pts_pg_away_team,season_win_pct_away_team,season_win_pct_at_home_away_team,season_away_win_pct_away_team
0,24600052,1610612744,1610612738,21946,Regular Season,W,L,66.0,54.0,12,-12,0,1946-11-26,NaN,1946,Regulation,Tuesday,11,Golden State Warriors,GSW,Golden State,California,1946,Chase Center,18064.0,16.803337,96.178660,Boston Celtics,BOS,Boston,Massachusetts,1946,TD Garden,18624.0,42.355433,-71.060511,NaN,8.0,2.0,6.0,0.0,2.0,2.0,10.0,6.0,4.0,408.0,259.0,66.7,68.0,64.8,0.800,1.000,0.500,NaN,3.0,6.0,2.0,1.0,1.0,5.0,9.0,3.0,6.0,186.0,311.0,55.2,62.0,51.8,0.333,0.667,0.167
1,24600063,1610612752,1610612744,21946,Regular Season,W,L,64.0,60.0,4,-4,0,1946-11-30,NaN,1946,Regulation,Saturday,11,New York Knicks,NYK,New York,New York,1946,Madison Square Garden,19763.0,40.712728,-74.006015,Golden State Warriors,GSW,Golden State,California,1946,Chase Center,18064.0,16.803337,96.178660,NaN,2.0,5.0,2.0,2.0,0.0,3.0,7.0,4.0,3.0,234.0,178.0,58.9,58.5,59.3,0.286,0.500,0.000,NaN,8.0,2.0,6.0,0.0,2.0,2.0,10.0,6.0,4.0,408.0,259.0,66.7,68.0,64.8,0.800,1.000,0.500
2,24600083,1610612752,1610612738,21946,Regular Season,W,L,62.0,44.0,18,-18,0,1946-12-08,NaN,1946,Regulation,Sunday,12,New York Knicks,NYK,New York,New York,1946,Madison Square Garden,19763.0,40.712728,-74.006015,Boston Celtics,BOS,Boston,Massachusetts,1946,TD Garden,18624.0,42.355433,-71.060511,NaN,2.0,5.0,2.0,2.0,0.0,3.0,7.0,4.0,3.0,234.0,178.0,58.9,58.5,59.3,0.286,0.500,0.000,NaN,3.0,6.0,2.0,1.0,1.0,5.0,9.0,3.0,6.0,186.0,311.0,55.2,62.0,51.8,0.333,0.667,0.167
3,24600095,1610612738,1610612744,21946,Regular Season,W,L,77.0,65.0,12,-12,0,1946-12-14,NaN,1946,Regulation,Saturday,12,Boston Celtics,BOS,Boston,Massachusetts,1946,TD Garden,18624.0,42.355433,-71.060511,Golden State Warriors,GSW,Golden State,California,1946,Chase Center,18064.0,16.803337,96.178660,NaN,3.0,6.0,2.0,1.0,1.0,5.0,9.0,3.0,6.0,186.0,311.0,55.2,62.0,51.8,0.333,0.667,0.167,NaN,8.0,2.0,6.0,0.0,2.0,2.0,10.0,6.0,4.0,408.0,259.0,66.7,68.0,64.8,0.800,1.000,0.500
4,24600119,1610612738,1610612744,21946,Regular Season,L,W,60.0,63.0,-3,3,0,1946-12-27,NaN,1946,Regulation,Friday,12,Boston Celtics,BOS,Boston,Massachusetts,1946,TD Garden,18624.0,42.355433,-71.060511,Golden State Warriors,GSW,Golden State,California,1946,Chase Center,18064.0,16.803337,96.178660,NaN,3.0,6.0,2.0,1.0,1.0,5.0,9.0,3.0,6.0,186.0,311.0,55.2,62.0,51.8,0.333,0.667,0.167,NaN,8.0,2.0,6.0,0.0,2.0,2.0,10.0,6.0,4.0,408.0,259.0,66.7,68.0,64.8,0.800,1.000,0.500


In [186]:
combined_df.shape

(56362, 72)

#### Additional Feature Engineering
- Distance between teams (in miles) using city coordinates calculated earlier 
- Evaluating team performance previous season
- Creating attendance related classifiers 


##### Distance between teams

In [188]:
# Apply the distance function to calculate the distance for each game
combined_df['distance_miles'] = combined_df.apply(calculate_distance, axis=1)

# Print sample results
print(combined_df[['game_id', 'full_name_home_team', 'city_home_team', 'full_name_away_team', 'city_away_team', 'distance_miles']].head())

    game_id    full_name_home_team city_home_team    full_name_away_team city_away_team  distance_miles
0  24600052  Golden State Warriors   Golden State         Boston Celtics         Boston     8280.107619
1  24600063        New York Knicks       New York  Golden State Warriors   Golden State     8423.892467
2  24600083        New York Knicks       New York         Boston Celtics         Boston      190.193347
3  24600095         Boston Celtics         Boston  Golden State Warriors   Golden State     8280.107619
4  24600119         Boston Celtics         Boston  Golden State Warriors   Golden State     8280.107619


##### Previous Season Stats
- Identifying which teams made the playoffs the previous season

Playoff Teams

In [ ]:
# Extracting the relevant team_id column from the team_df
#team_ids = team_df['team_id'].unique()

In [189]:
# Create a DataFrame with all combinations of team_ids and seasons
# Using team_ids saved from earlier
all_seasons = combined_df['season'].unique()
all_teams_seasons = pd.MultiIndex.from_product([team_ids, all_seasons], names=['team', 'season']).to_frame(index=False)

In [190]:
# Create new column to merge prev season data
all_teams_seasons['merge_season'] = all_teams_seasons['season'] + 1
all_teams_seasons.head()

,team,season,merge_season
0,1610612737,1946,1947
1,1610612737,1947,1948
2,1610612737,1948,1949
3,1610612737,1949,1950
4,1610612737,1950,1951


In [191]:
# Filter the DataFrame to include only regular season and playoff games
filtered_df = combined_df[combined_df['season_type'].isin(['Regular', 'Playoffs'])]

# Display the filtered DataFrame to verify
filtered_df.head()

,game_id,team_id_home,team_id_away,season_id,season_type,wl_home,wl_away,pts_home,pts_away,plus_minus_home,plus_minus_away,min,game_date,attendance,season,game_duration,day_of_week,month,full_name_home_team,abbreviation_home_team,city_home_team,state_home_team,year_founded_home_team,arena_home_team,arenacapacity_home_team,latitude_home_team,longitude_home_team,full_name_away_team,abbreviation_away_team,city_away_team,state_away_team,year_founded_away_team,arena_away_team,arenacapacity_away_team,latitude_away_team,longitude_away_team,avg_attendance_home_team,season_w_home_team,season_l_home_team,wins_at_home_home_team,losses_at_home_home_team,away_wins_home_team,away_losses_home_team,games_this_season_home_team,games_at_home_home_team,away_games_home_team,total_pts_at_home_home_team,total_away_pts_home_team,season_pts_pg_home_team,pts_pg_at_home_home_team,away_pts_pg_home_team,season_win_pct_home_team,season_win_pct_at_home_home_team,season_away_win_pct_home_team,avg_attendance_away_team,season_w_away_team,season_l_away_team,wins_at_home_away_team,losses_at_home_away_team,away_wins_away_team,away_losses_away_team,games_this_season_away_team,games_at_home_away_team,away_games_away_team,total_pts_at_home_away_team,total_away_pts_away_team,season_pts_pg_away_team,pts_pg_at_home_away_team,away_pts_pg_away_team,season_win_pct_away_team,season_win_pct_at_home_away_team,season_away_win_pct_away_team,distance_miles
13,44600211,1610612744,1610612752,41946,Playoffs,W,L,82.0,70.0,12,-12,0,1947-04-12,NaN,1946,Regulation,Saturday,4,Golden State Warriors,GSW,Golden State,California,1946,Chase Center,18064.0,16.803337,96.178660,New York Knicks,NYK,New York,New York,1946,Madison Square Garden,19763.0,40.712728,-74.006015,NaN,2.0,0.0,1.0,0.0,1.0,0.0,2.0,1.0,1.0,82.0,72.0,77.0,82.0,72.0,1.000,1.0,1.0,4607.0,0.0,2.0,0.0,1.0,0.0,1.0,2.0,1.0,1.0,53.0,70.0,61.5,53.0,70.0,0.000,0.0,0.0,8423.892467
14,44600212,1610612752,1610612744,41946,Playoffs,L,W,53.0,72.0,-19,19,0,1947-04-14,4607.0,1946,Regulation,Monday,4,New York Knicks,NYK,New York,New York,1946,Madison Square Garden,19763.0,40.712728,-74.006015,Golden State Warriors,GSW,Golden State,California,1946,Chase Center,18064.0,16.803337,96.178660,4607.0,0.0,2.0,0.0,1.0,0.0,1.0,2.0,1.0,1.0,53.0,70.0,61.5,53.0,70.0,0.000,0.0,0.0,NaN,2.0,0.0,1.0,0.0,1.0,0.0,2.0,1.0,1.0,82.0,72.0,77.0,82.0,72.0,1.000,1.0,1.0,8423.892467
106,44800312,1610612747,1610612758,41948,Playoffs,W,L,67.0,55.0,12,-12,0,1949-03-29,10412.0,1948,Regulation,Tuesday,3,Los Angeles Lakers,LAL,Los Angeles,California,1948,Crypto.com Arena,19060.0,34.053691,-118.242766,Sacramento Kings,SAC,Sacramento,California,1948,Golden 1 Center,17500.0,38.581061,-121.493895,10412.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,67.0,0.0,67.0,67.0,0.0,1.000,1.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,55.0,55.0,0.0,55.0,0.000,0.0,0.0,360.983358
209,44900231,1610612755,1610612744,41949,Playoffs,W,L,93.0,76.0,17,-17,0,1950-03-22,NaN,1949,Regulation,Wednesday,3,Philadelphia 76ers,PHI,Philadelphia,Pennsylvania,1949,Wells Fargo Center,20478.0,39.952724,-75.163526,Golden State Warriors,GSW,Golden State,California,1946,Chase Center,18064.0,16.803337,96.178660,NaN,5.0,4.0,4.0,0.0,1.0,4.0,9.0,4.0,5.0,358.0,376.0,81.6,89.5,75.2,0.556,1.0,0.2,NaN,0.0,2.0,0.0,1.0,0.0,1.0,2.0,1.0,1.0,53.0,76.0,64.5,53.0,76.0,0.000,0.0,0.0,8486.518317
210,44900232,1610612744,1610612755,41949,Playoffs,L,W,53.0,59.0,-6,6,0,1950-03-23,NaN,1949,Regulation,Thursday,3,Golden State Warriors,GSW,Golden State,California,1946,Chase Center,18064.0,16.803337,96.178660,Philadelphia 76ers,PHI,Philadelphia,Pennsylvania,1949,Wells Fargo Center,20478.0,39.952724,-75.163526,NaN,0.0,2.0,0.0,1.0,0.0,1.0,2.0,1.0,1.0,53.0,76.0,64.5,53.0,76.0,0.000,0.0,0.0,NaN,5.0,4.0,4.0,0.0,1.0,4.0,9.0,4.0,5.0,358.0,376.0,81.6,89.5,75.2,0.556,1.0,0.2,8486.518317


In [192]:
# Determine playoff participation for home teams
home_playoffs = filtered_df.groupby(['team_id_home', 'season'])['season_type'].apply(lambda x: 'Playoffs' in x.values).reset_index()
home_playoffs.rename(columns={'team_id_home': 'team', 'season_type': 'made_playoffs_home'}, inplace=True)

# Determine playoff participation for away teams
away_playoffs = filtered_df.groupby(['team_id_away', 'season'])['season_type'].apply(lambda x: 'Playoffs' in x.values).reset_index()
away_playoffs.rename(columns={'team_id_away': 'team', 'season_type': 'made_playoffs_away'}, inplace=True)

# Merge the home and away playoff statuses
playoff_teams = pd.merge(home_playoffs, away_playoffs, on=['team', 'season'], how='outer')

# Display the playoff_teams DataFrame
playoff_teams.head()

,team,season,made_playoffs_home,made_playoffs_away
0,1610612737,1955,True,True
1,1610612737,1956,True,True
2,1610612737,1959,True,True
3,1610612737,1962,True,True
4,1610612737,1963,True,True


In [193]:
# Fill NaN values with 0 and convert to boolean
playoff_teams['made_playoffs_home'] = playoff_teams['made_playoffs_home'].fillna(0).astype(bool)
playoff_teams['made_playoffs_away'] = playoff_teams['made_playoffs_away'].fillna(0).astype(bool)

In [194]:
# Combine the home and away playoff statuses
playoff_teams['prev_playoffs'] = playoff_teams[['made_playoffs_home', 'made_playoffs_away']].max(axis=1)

# Drop the intermediate columns
playoff_teams = playoff_teams.drop(columns=['made_playoffs_home', 'made_playoffs_away'])

# Display the playoff_teams DataFrame
playoff_teams.head()

,team,season,prev_playoffs
0,1610612737,1955,True
1,1610612737,1956,True
2,1610612737,1959,True
3,1610612737,1962,True
4,1610612737,1963,True


In [195]:
# Merge all teams and seasons with the playoff status
all_teams_seasons = pd.merge(all_teams_seasons, playoff_teams, on=['team', 'season'], how='left')

# Fill NaN values with False
all_teams_seasons['prev_playoffs'] = all_teams_seasons['prev_playoffs'].fillna(False).astype(bool)

# Display the all_teams_seasons DataFrame
all_teams_seasons.head()

/var/folders/7z/gr96tjyj6gb84p0c2xk4xfr40000gn/T/ipykernel_19064/2003544979.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  all_teams_seasons['prev_playoffs'] = all_teams_seasons['prev_playoffs'].fillna(False).astype(bool)


,team,season,merge_season,prev_playoffs
0,1610612737,1946,1947,False
1,1610612737,1947,1948,False
2,1610612737,1948,1949,False
3,1610612737,1949,1950,False
4,1610612737,1950,1951,False


In [196]:
# Drop season column
all_teams_seasons.drop('season', axis=1, inplace=True)

In [198]:
combined_df.shape

(56362, 73)

In [199]:
# Merge to add all home team data
combined_df = pd.merge(combined_df, all_teams_seasons, 
                               left_on=['team_id_home', 'season'], right_on=['team', 'merge_season'], suffixes=('', '_home_team'))

# Drop the redundant team_id column from the home merge
combined_df.drop('team', axis=1, inplace=True)
# Drop the redundant prev_season column from the home merge
combined_df.drop('merge_season', axis=1, inplace=True)

# Merge to add all away team data
combined_df = pd.merge(combined_df, all_teams_seasons, 
                           left_on=['team_id_away', 'season'], right_on=['team', 'merge_season'], suffixes=('_home_team', '_away_team'))

# Drop the redundant team_id column from the away merge
combined_df.drop('team', axis=1, inplace=True)
# Drop the redundant prev_season column from the away merge
combined_df.drop('merge_season', axis=1, inplace=True)

# Updated dataframe
combined_df.head()

,game_id,team_id_home,team_id_away,season_id,season_type,wl_home,wl_away,pts_home,pts_away,plus_minus_home,plus_minus_away,min,game_date,attendance,season,game_duration,day_of_week,month,full_name_home_team,abbreviation_home_team,city_home_team,state_home_team,year_founded_home_team,arena_home_team,arenacapacity_home_team,latitude_home_team,longitude_home_team,full_name_away_team,abbreviation_away_team,city_away_team,state_away_team,year_founded_away_team,arena_away_team,arenacapacity_away_team,latitude_away_team,longitude_away_team,avg_attendance_home_team,season_w_home_team,season_l_home_team,wins_at_home_home_team,losses_at_home_home_team,away_wins_home_team,away_losses_home_team,games_this_season_home_team,games_at_home_home_team,away_games_home_team,total_pts_at_home_home_team,total_away_pts_home_team,season_pts_pg_home_team,pts_pg_at_home_home_team,away_pts_pg_home_team,season_win_pct_home_team,season_win_pct_at_home_home_team,season_away_win_pct_home_team,avg_attendance_away_team,season_w_away_team,season_l_away_team,wins_at_home_away_team,losses_at_home_away_team,away_wins_away_team,away_losses_away_team,games_this_season_away_team,games_at_home_away_team,away_games_away_team,total_pts_at_home_away_team,total_away_pts_away_team,season_pts_pg_away_team,pts_pg_at_home_away_team,away_pts_pg_away_team,season_win_pct_away_team,season_win_pct_at_home_away_team,season_away_win_pct_away_team,distance_miles,prev_playoffs_home_team,prev_playoffs_away_team
0,24700006,1610612738,1610612744,21947,Regular Season,L,W,74.0,79.0,-5,5,0,1947-11-14,NaN,1947,Regulation,Friday,11,Boston Celtics,BOS,Boston,Massachusetts,1946,TD Garden,18624.0,42.355433,-71.060511,Golden State Warriors,GSW,Golden State,California,1946,Chase Center,18064.0,16.803337,96.178660,NaN,4.0,9.0,2.0,4.0,2.0,5.0,13.0,6.0,7.0,413.0,438.0,65.5,68.8,62.6,0.308,0.333,0.286,NaN,8.0,7.0,3.0,4.0,5.0,3.0,15.0,7.0,8.0,471.0,554.0,68.3,67.3,69.2,0.533,0.429,0.625,8280.107619,False,True
1,24700020,1610612752,1610612744,21947,Regular Season,L,W,78.0,83.0,-5,5,0,1947-11-22,NaN,1947,Regulation,Saturday,11,New York Knicks,NYK,New York,New York,1946,Madison Square Garden,19763.0,40.712728,-74.006015,Golden State Warriors,GSW,Golden State,California,1946,Chase Center,18064.0,16.803337,96.178660,NaN,9.0,5.0,4.0,4.0,5.0,1.0,14.0,8.0,6.0,563.0,471.0,73.9,70.4,78.5,0.643,0.500,0.833,NaN,8.0,7.0,3.0,4.0,5.0,3.0,15.0,7.0,8.0,471.0,554.0,68.3,67.3,69.2,0.533,0.429,0.625,8423.892467,True,True
2,24700024,1610612738,1610612752,21947,Regular Season,L,W,75.0,91.0,-16,16,0,1947-11-25,NaN,1947,Regulation,Tuesday,11,Boston Celtics,BOS,Boston,Massachusetts,1946,TD Garden,18624.0,42.355433,-71.060511,New York Knicks,NYK,New York,New York,1946,Madison Square Garden,19763.0,40.712728,-74.006015,NaN,4.0,9.0,2.0,4.0,2.0,5.0,13.0,6.0,7.0,413.0,438.0,65.5,68.8,62.6,0.308,0.333,0.286,NaN,9.0,5.0,4.0,4.0,5.0,1.0,14.0,8.0,6.0,563.0,471.0,73.9,70.4,78.5,0.643,0.500,0.833,190.193347,False,True
3,24700026,1610612752,1610612738,21947,Regular Season,L,W,63.0,65.0,-2,2,0,1947-11-26,NaN,1947,Regulation,Wednesday,11,New York Knicks,NYK,New York,New York,1946,Madison Square Garden,19763.0,40.712728,-74.006015,Boston Celtics,BOS,Boston,Massachusetts,1946,TD Garden,18624.0,42.355433,-71.060511,NaN,9.0,5.0,4.0,4.0,5.0,1.0,14.0,8.0,6.0,563.0,471.0,73.9,70.4,78.5,0.643,0.500,0.833,NaN,4.0,9.0,2.0,4.0,2.0,5.0,13.0,6.0,7.0,413.0,438.0,65.5,68.8,62.6,0.308,0.333,0.286,190.193347,True,False
4,24700028,1610612744,1610612752,21947,Regular Season,L,W,59.0,81.0,-22,22,0,1947-11-27,NaN,1947,Regulation,Thursday,11,Golden State Warriors,GSW,Golden State,California,1946,Chase Center,18064.0,16.803337,96.178660,New York Knicks,NYK,New York,New York,1946,Madison Square Garden,19763.0,40.712728,-74.006015,NaN,8.0,7.0,3.0,4.0,5.0,3.0,15.0,7.0,8.0,471.0,554.0,68.3,67.3,69.2,0.533,0.429,0.625,NaN,9.0,5.0,4.0,4.0,5.0,1.0,14.0,8.0,6.0,563.0,471.0,73.9,70.4,78.5,0.643,0.500,0.833,8423.892467,True,True


In [200]:
# Merging effectively drops first season games with no previous season stats available
combined_df.shape

(56323, 75)

##### Attendance Metrics
- Defining sell-out games (since I'm missing ticket sales, I'm using attendance to roughly calculate which games were sold likley "sold-out")
- Calculating average attendance each season and classifying each games attendance as above or below average 

Attendance Ratio and Classifier

In [205]:
# Calculate the attendance ratio using .loc
combined_df.loc[:, 'attendance_ratio'] = combined_df['attendance'] / combined_df['arenacapacity_home_team']

# Defining near sell-out as 90% or higher attendance using .loc
combined_df.loc[:, 'near_sell_out'] = (combined_df['attendance_ratio'] >= 0.95).astype(int)

# Display the first few rows to check the result
combined_df[['attendance', 'arenacapacity_home_team', 'attendance_ratio', 'near_sell_out']].tail()

,attendance,arenacapacity_home_team,attendance_ratio,near_sell_out
56318,19528.0,19520.0,1.000410,1
56319,19537.0,19520.0,1.000871,1
56320,20019.0,19600.0,1.021378,1
56321,20184.0,19600.0,1.029796,1
56322,19537.0,19520.0,1.000871,1


Above average attendance classifier

In [206]:
# Define new classification column for games with above average attendance
combined_df.loc[:, 'above_avg_attendance'] = (combined_df['attendance'] > combined_df['avg_attendance_home_team']).astype(int)

# Display the last few rows to check the result
combined_df[['attendance', 'arenacapacity_home_team', 'avg_attendance_home_team', 'above_avg_attendance']].tail()

,attendance,arenacapacity_home_team,avg_attendance_home_team,above_avg_attendance
56318,19528.0,19520.0,19646.0,0
56319,19537.0,19520.0,19646.0,0
56320,20019.0,19600.0,19942.0,1
56321,20184.0,19600.0,19942.0,1
56322,19537.0,19520.0,19646.0,0


## Exporting All Data

In [ ]:
# # Optional
# # Export team location info 
# team_location_df.to_csv('data/team_location.csv', index=False)

# # Export transformed team and game data 
# game_df.to_csv('data/games.csv', index=False)
# team_df.to_csv('data/teams.csv', index=False)

# # Export new season stats data
# season_stats.to_csv('data/season_stats.csv', index=False)

Dropping columns
- arena_away_team
- arenacapacity_away_team
- avg_attendance_away_team
- min

Reordering for readablity 

In [ ]:
combined_df.shape

In [209]:
# Reorder and dropping columns
combined_df = combined_df[['game_id', 'team_id_home', 'team_id_away', 'full_name_home_team', 'full_name_away_team', 'wl_home', 'wl_away',
                             'game_date', 'day_of_week', 'month', 'game_duration',
                             'season_id', 'season', 'season_type',
                             'attendance', 'avg_attendance_home_team', 'above_avg_attendance', 'arenacapacity_home_team', 'attendance_ratio', 'near_sell_out',  
                             'pts_home', 'pts_away', 'plus_minus_home', 'plus_minus_away', 
                             'abbreviation_home_team', 'city_home_team', 'state_home_team', 'year_founded_home_team', 'arena_home_team', 'latitude_home_team', 'longitude_home_team', 'distance_miles',
                             'abbreviation_away_team', 'city_away_team', 'state_away_team', 'year_founded_away_team', 'latitude_away_team', 'longitude_away_team',  
                             'season_w_home_team', 'season_l_home_team', 'wins_at_home_home_team', 'losses_at_home_home_team', 'away_wins_home_team', 'away_losses_home_team', 'games_this_season_home_team', 'games_at_home_home_team', 'away_games_home_team', 'total_pts_at_home_home_team', 'total_away_pts_home_team', 'season_pts_pg_home_team','pts_pg_at_home_home_team', 'away_pts_pg_home_team', 'season_win_pct_home_team', 'season_win_pct_at_home_home_team', 'season_away_win_pct_home_team', 
                             'season_w_away_team', 'season_l_away_team', 'wins_at_home_away_team', 'losses_at_home_away_team', 'away_wins_away_team', 'away_losses_away_team', 'games_this_season_away_team', 'games_at_home_away_team', 'away_games_away_team', 'total_pts_at_home_away_team', 'total_away_pts_away_team', 'season_pts_pg_away_team', 'pts_pg_at_home_away_team', 'away_pts_pg_away_team', 'season_win_pct_away_team', 'season_win_pct_at_home_away_team', 'season_away_win_pct_away_team',  
                             'prev_playoffs_home_team', 'prev_playoffs_away_team']]
combined_df.shape

In [210]:
# Must export combined data for further analysis and modeling
combined_df.to_csv('data/combined_data.csv', index=False)

## Close Database Connection
- Ensure connection has been closed if not closed earlier

In [9]:
# Double check connection is closed
close_connection()